Q3 (scatterplot)
idea 1:
- scatterplot
- eix x: school
- eix y: mass shootings
- color: year
- point: state in year

idea2:
- violin plot
- violin: state
- distribucio violin: years

idea3:
- scatterplot de totals, sense separar per anys


county, state, shootings, school, year

In [17]:
import pandas as pd
import altair as alt

df_shootings = pd.read_csv("datasets/MassShootingsComplete_FIPS.csv")
df_school_incidents = pd.read_csv("datasets/SchoolIncidents_FIPS.csv")
df_population = pd.read_csv("datasets/CountyPopulationAllYears.csv")

In [ ]:

total_shootings_per_state = df_shootings.groupby('State').size().reset_index(name='Total Shootings')


# Calcula el total d'incidents escolars per estat
total_school_incidents_per_state = df_school_incidents.groupby('State').size().reset_index(name='Total School Incidents')

# Uneix els dos datasets basant-te en l'estat
merged_data = pd.merge(total_shootings_per_state, total_school_incidents_per_state, on='State')

# Crea el scatterplot amb Altair
scatterplot = alt.Chart(merged_data).mark_circle(size=100).encode(
    x=alt.Y('Total Shootings:Q', title='Total Shootings', scale=alt.Scale(type='log')),
    y=alt.X('Total School Incidents:Q', title='Total School Incidents', scale=alt.Scale(type='log')),
    tooltip=['State', 'Total Shootings', 'Total School Incidents'],  # Afegeix informació de descripció
).properties(
    title='Comparison of Total Shootings and School Incidents by State'
)

scatterplot.show()

alt.Chart(...)

In [14]:
# Converteix les dates a un format de data de pandas

df_shootings['Incident Date'] = pd.to_datetime(df_shootings['Incident Date'])
df_school_incidents['Incident Date'] = pd.to_datetime(df_school_incidents['Incident Date'])

# Extreu l'any de la data de l'incident per ambdós datasets
df_shootings['Year'] = df_shootings['Incident Date'].dt.year
df_school_incidents['Year'] = df_school_incidents['Incident Date'].dt.year

# Filtra només els anys 2022 i posteriors
df_shootings = df_shootings[df_shootings['Year'] >= 2022]
df_school_incidents = df_school_incidents[df_school_incidents['Year'] >= 2022]

# Calcula el total de tirotejos per estat i any
total_shootings_per_state_year = df_shootings.groupby(['State', 'Year']).size().reset_index(name='Total Shootings')

# Calcula el total d'incidents escolars per estat i any
total_school_incidents_per_state_year = df_school_incidents.groupby(['State', 'Year']).size().reset_index(name='Total School Incidents')

# Uneix els dos datasets per estat i any
merged_data = pd.merge(total_shootings_per_state_year, total_school_incidents_per_state_year, on=['State', 'Year'])

# Crea el scatterplot amb Altair
scatterplot = alt.Chart(merged_data).mark_circle(size=100).encode(
    x=alt.X('Total Shootings:Q', title='Total Shootings', scale=alt.Scale(type='log')),
    y=alt.Y('Total School Incidents:Q', title='Total School Incidents', scale=alt.Scale(type='log')),
    color=alt.Color('Year:O', 
                    title='Year', 
                    scale=alt.Scale(
                        domain=[2022, 2023],  # Anys que tens en les dades
                        range=['#f1c40f', '#9b59b6']  # Paleta de colors contrastada
                    )
    ),
    tooltip=['State', 'Year', 'Total Shootings', 'Total School Incidents'],  # Afegeix informació de descripció
).properties(
    title='Comparison of Total Shootings and School Incidents by State (2022 onwards)'
)


scatterplot.show()

alt.Chart(...)